In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지에 들어가는 한글을 제대로 보기 위해 한글 폰트 적용
import platform
font_dict = {
    'Linux': 'Noto Sans CJK KR',
    'Darwin': 'Apple SD Gothic Neo', # macOS
    'Windows': 'Malgun Gothic' # MS-Windows
}
try:
    mpl.rc('font', family=font_dict[platform.system()])
except:
    pass
mpl.rc('axes', unicode_minus=False)

%matplotlib inline

# [All about 따릉이 EDA, 5편] 마포구, 따릉이는 얼마나 어떻게 이용되고 있을까? by 흠시

**출처:** https://dailyheumsi.tistory.com/91

**데이터:** 서울특별시 공공자전거 대여이력 정보 @ [서울 열린데이터 광장](https://data.seoul.go.kr)
  - 자전거 이동경로에 대한 데이터 분석이 가능하도록 년도별, 대여소별, 자전거별 대여이력 원천 데이터를 제공
  - https://data.seoul.go.kr/dataList/OA-15182/F/1/datasetView.do
  - `서울특별시 공공자전거 대여정보_201905.csv` (용량 299.3MB, 수정일 2019.06.15)

> [흠시] 이번 글에서는, 서울 시내 지역구 중, **15년도부터 꾸준하게 따릉이 이용률이 높았던 지역인 마포구**에 대해 좀 더 자세히 알아보려 한다.  
저번(3편)과 마찬가지로 **19년 5월 데이터**를 통해 살펴본다.  
(\*9월은 1년 중, 따릉이 이용량이 가장 높은 달이다. 대표젹인 패턴을 가장 잘 보여줄 수 있는 기간이라 생각하여 9월 한 달만을 표본으로 선택하였다.)

In [ ]:
%%time

# 3편에서 사용했던 압축된 대여정보 데이터를 불러온다.
#   서울특별시 공공자전거 대여정보_201905.csv.gz (76.3MB)

from pathlib import Path

데이터_폴더 = Path('../data')
공공자전거_대여정보_201905 = 데이터_폴더 / '서울특별시 공공자전거 대여정보_201905.csv.gz'

# 데이터 파일의 첫 줄에 헤더 정보가 없으므로, 컬럼 명을 직접 넣어주어야 한다.
df = pd.read_csv(공공자전거_대여정보_201905,
    encoding='cp949',
    names=['자전거번호',
           '대여일시', '대여대여소번호', '대여대여소명', '대여거치대',
           '반납일시', '반납대여소번호', '반납대여소명', '반납거치대',
           '이용시간', '이용거리'],
    parse_dates=['대여일시', '반납일시']
)
df.info(memory_usage='deep')

In [ ]:
# 1편에서 사용했던 대여소 정보 데이터도 불러오자. 이번에는 대여소명도 사용한다.

공공자전거_대여소_정보_데이터 = 데이터_폴더 / '서울특별시 공공자전거 대여소 정보(19.12.9).xlsx'

rental = pd.read_excel(공공자전거_대여소_정보_데이터,
    index_col='대여소ID',
    usecols=['대여소_구', '대여소명', '대여소ID', '위도', '경도'],
    dtype={"대여소_구": "category"},
    skipfooter=1
)
rental.info()

In [ ]:
# 대여대여소번호(대여소ID)와 반납대여소번호에 맞게 대여지역을 가져오자.
df = (df
    .merge(rental['대여소_구'], left_on='대여대여소번호', right_index=True)
    .rename(columns={'대여소_구': '대여지역'})
    .merge(rental['대여소_구'], left_on='반납대여소번호', right_index=True)
    .rename(columns={'대여소_구': '반납지역'})
)

# 대여일시와 반납일시에서 날짜(day), 요일(dayofweek), 시간(hour)을 분리하자.
df['대여일'] = df['대여일시'].dt.day
df['대여요일'] = df['대여일시'].dt.dayofweek
df['대여시간'] = df['대여일시'].dt.hour
df['반납시간'] = df['반납일시'].dt.hour

df.info(memory_usage='deep')

---
## 1. 이용 수치량 살펴보기

> [흠시] 가장 먼저, 수치적으로 쉽게 살펴볼 수 있는 것부터 보자.  
시간에 따른 이용량 패턴, 이용거리, 시간 등이다.

### 1.1. 시간대별 사용량

In [ ]:
# 마포구 평일(월-금) 시간대별 대여횟수(일) [Series]
# 월(0), 화(1), 수(2), 목(3), 금(4), 토(5), 일(6)

by_hour_rental = df[
    (df['대여요일'] < 5) & (df['대여지역'] == '마포구')
].groupby('대여시간').size() / 5
by_hour_rental

In [ ]:
# 마포구 평일(월-금) 시간대별 반납횟수(일) [Series]
# 월(0), 화(1), 수(2), 목(3), 금(4), 토(5), 일(6)

by_hour_return = df[
    (df['대여요일'] < 5) & (df['반납지역'] == '마포구')
].groupby('반납시간').size() / 5
by_hour_return

In [ ]:
# 대여횟수(Series)와 반납횟수(Series)를 합쳐 하나의 DataFrame으로 만들자.

by_hour = pd.DataFrame(data={
    '대여': by_hour_rental,
    '반납': by_hour_return
})
by_hour

In [ ]:
ax = by_hour.plot.line(
    style='.-',
    rot=0,
    xticks=range(0, 24),
    xlabel="시",
    title="평일, 시간대별 사용량(대여, 반납)"
)
ax.set_frame_on(False)
ax.legend(frameon=False);

In [ ]:
# 마포구 주말(토-일) 시간대별 대여횟수(일) [Series]
by_hour_rental = df[
    (df['대여요일'] >= 5) & (df['대여지역'] == '마포구')
].groupby('대여시간').size() / 2

# 마포구 주말(토-일) 시간대별 반납횟수(일) [Series]
by_hour_return = df[
    (df['대여요일'] >= 5) & (df['반납지역'] == '마포구')
].groupby('반납시간').size() / 2

by_hour = pd.DataFrame(data={
    '대여': by_hour_rental,
    '반납': by_hour_return
})

ax = by_hour.plot.line(
    style='.-',
    rot=0,
    xticks=range(0, 24),
    xlabel="시",
    title="주말, 시간대별 사용량(대여, 반납)"
)
ax.set_frame_on(False)
ax.legend(frameon=False);

In [ ]:
# 위의 두 그래프를 한꺼번에 그려보자.

_, axes = plt.subplots(ncols=2, figsize=(15, 4))

for ax, dayofweek, days in zip(axes, ['평일', '주말'], [5, 2]):   
    if dayofweek == '평일':
        _df = df[df['대여요일'] < 5]
    else:
        _df = df[df['대여요일'] >= 5]

    by_hour_rental = _df[_df['대여지역'] == '마포구'].groupby('대여시간').size() / days
    by_hour_return = _df[_df['반납지역'] == '마포구'].groupby('반납시간').size() / days
    by_hour = pd.DataFrame(data={
        '대여': by_hour_rental,
        '반납': by_hour_return
    })

    by_hour.plot.line(
        style='.-',
        rot=0,
        xticks=range(24),
        xlabel="시",
        yticks=range(0, 3500, 500),
        title=f"{dayofweek}, 시간대별 사용량(대여, 반납)",
        ax=ax
    )
    ax.set_frame_on(False)
    ax.legend(frameon=False)

> [흠시] 평일의 경우, 아침 8시와 저녁 18시에 고점을 찍는 패턴은, **일반적인 서울시 따릉이 패턴과 같다.**  
한편, 아침 8시엔 반납량이 대여량보다 더 많고, 저녁 18시에는 대여량이 더 많은 걸로 보아서,  
마포구가 아닌 **다른 지역에서 마포구로 출퇴근하는 사람들이 많다는걸 짐작**할 수 있다.

> [흠시] 주말의 경우, 전 시간대에서 대여/반냡량이 비슷함을 알 수 있다.

### 2.2 시간대별 이용거리, 이용시간

In [ ]:
# 마포구에서 대여한 시간별로 이용거리와 이용시간의 중간값을 구해보자.

_, axes = plt.subplots(ncols=2, figsize=(15, 4))

for ax, var, unit in zip(axes, ['이용거리', '이용시간'], ['m', '분']):
    _df = df[df['대여지역'] == '마포구']

    by_hour_rental_weekday = _df[_df['대여요일'] < 5].groupby('대여시간')[var].median()
    by_hour_rental_weekend = _df[_df['대여요일'] >= 5].groupby('대여시간')[var].median()
    by_hour = pd.DataFrame(data={
        '평일': by_hour_rental_weekday,
        '주말': by_hour_rental_weekend
    })
    
    by_hour.plot.line(
        style='.-',
        rot=0,
        color={'평일': 'C3', '주말': 'C4'},
        xticks=range(24),
        xlabel="시",
        title=f"시간대별 중간값 {var} ({unit})",
        ax=ax
    )
    ax.set_frame_on(False)
    ax.legend(frameon=False)
    
    print(
        f"[{var}] "
        f"평일 중간값 평균: {by_hour_rental_weekday.mean():.0f}{unit}, "
        f"주말 중간값 평균: {by_hour_rental_weekend.mean():.0f}{unit} "
    )

> [흠시] 먼저 이용거리, 이용시간 둘다 **평일과 주말의 패턴이 크게 다르지 않음**을 알 수 있다.  
전체적인 량으로 보면, 주말이 높다. 이는 서울시 전체 따릉이 패턴과 동일하다.  
또한, 평일과 주말 모두 다, 16시 - 22시 사이에 이용량이 많은걸 볼 수 있다.

---
## 2. 주로 어떤 사람들이 이용했을까?

> [흠시] 이번에는 유저 분석이다.  
이전 포스팅과 같이 기준을 성별, 연령대 순으로 살펴본다.

In [ ]:
%%time

# 4편에서 저장한 시간대별 이용정보 데이터를 불러오자.

users = pd.read_pickle(데이터_폴더 / '서울특별시 공공자전거 이용정보(시간대별).pkl.gz')
users.info(memory_usage='deep')

In [ ]:
# 이 중 19년도 5월 마포구 데이터만 선택하자.

mapo_users = users[
    (users['대여소_구'] == '마포구') &
    (users['년'] == 2019) &
    (users['월'] == 5)
]
mapo_users.head()

### 2.1. 성별 기준

In [ ]:
# 마포구 성별 기준 이용량 [Series]

use_per_sex = mapo_users.groupby('성별')['이용건수'].sum()
use_per_sex

In [ ]:
sex_colors = ['crimson', 'royalblue']

_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 4))

use_per_sex.plot.bar(
    rot=0,
    color=sex_colors,
    title="남녀 이용량(19년 5월, 마포구)",
    ax=ax1
)
ax1.set_frame_on(False)
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{height:.0f}", (x+width/2, height+1000), ha='center')

use_per_sex_rate = use_per_sex / use_per_sex.sum()
use_per_sex_rate.plot.pie(
    autopct='%.1f%%',
    colors=sex_colors,
    title="남녀 이용비율(19년 5월, 마포구)",
    ax=ax2
)
ax2.set_frame_on(False)

> [흠시] 남자 이용량이 훨씬 많은건, 서울시 전체 패턴과 동일하다.  
다만, 이용비율을 보면 남녀 비율이 59:41로,  
서울시 전체에서는 62:37 이었던 것과 비교해보면, 다른 지역구보다 **여성비율이 조금 더 많은 편(4% 정도)이다.**

### 3.2. 연령대 기준

In [ ]:
mapo_users['연령대코드'].cat.categories

In [ ]:
# 연령대코드는 어떻게 정렬할 수 있을까?

age_order = sorted(
    mapo_users['연령대코드'].cat.categories,
    key=lambda x: x.replace('~', '')
)
age_order

In [ ]:
# 마포구 연령대 기준 이용량 [Series]

use_per_age = mapo_users.groupby('연령대코드').size()
use_per_age = use_per_age.reindex(age_order)
use_per_age

In [ ]:
_, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 4))

use_per_age.plot.bar(
    rot=0,
    color=[f"C{i}" for i in range(len(use_per_age))],
    title="연령대 이용량(19년 5월, 마포구)",
    ax=ax1
)
ax1.set_frame_on(False)
for p in ax1.patches:
    x, y, width, height = p.get_bbox().bounds
    ax1.annotate(f"{height:.0f}", (x+width/2, height+700), ha='center')

(use_per_age / use_per_age.sum()).plot.pie(
    autopct="%.1f%%",
#    colors=[f"C{i}" for i in range(len(use_per_age))],
    ylabel="",
    title="연령대 이용비율(19년 5월, 마포구)",
    ax=ax2
)
ax2.set_frame_on(False)

> [흠시] 전체적인 패턴은 서울시 전체 패턴과 다를게 없고, 연령대별 비율에서 조금 차이가 나보인다.  
굳이 시각화해서 보자면, 다음과 같다.

In [ ]:
# 서울시 전체 연령대 기준 이용량 [Series]

seoul_use_per_age = users.groupby('연령대코드').size()
seoul_use_per_age = seoul_use_per_age.reindex(age_order)
seoul_use_per_age

In [ ]:
# 서울시 전체와 마포구의 연령대 기준 이용비율을 구하자.

comp = pd.DataFrame({
    '서울시 전체': seoul_use_per_age,
    '마포구': use_per_age
})
comp = comp / comp.sum() * 100

# 차이를 구하고 차이 순으로 그림을 그리기 위해 정렬하자.
comp['diff'] = np.abs(comp['서울시 전체'] - comp['마포구'])
comp.sort_values('diff', inplace=True)
comp

In [ ]:
ax = comp.plot.barh(
    y=['서울시 전체', '마포구'],
    color={'서울시 전체': 'lightgrey', '마포구': 'grey'},
    title="서울시 전체 vs 마포구 연령대 비율 비교",
    figsize=(10, 4)
)
ax.set_frame_on(False)
ax.legend(frameon=False)
for p in ax.patches: 
    x, y, width, height = p.get_bbox().bounds 
    ax.text(width+0.5, y+height/2, f"{width:.1f}%", va='center')

> [흠시] 가장 크게 차이나는건 20대인데, 그래봐야 약 2% 차이밖에 안났다.  
나머지 연령대는 1% 미만이므로, 그다지 유의미한 차이는 아니라고 본다.  
즉, 서울시 전체 비율과 거의 같다고 볼 수 있다.

---
## 3. 대여소별로 사용자들이 다를까?

> [흠시] 이번에는 대여소 단위로 볼까 한다.  
살펴보고자 하는 것은, 남녀 이용자 비중이 크게 다른 대여소가 있는지,  
혹은 연령별로 주로 어디 대여소를 사용하고, 연령별로 차이가 있는지 등을 살펴본다.

### 3.1. 대여소별로 남녀비율이 많이 다를까?

> [흠시] 이를 위해, 대여소별 이용량을 남녀 기준으로 나눠서 시각화 해보자.

In [ ]:
# 먼저 마포구 대여소별 성별에 따른 이용량을 구하자.

df_pivot = mapo_users.pivot_table(
    index='대여소명', columns='성별', values='이용건수', aggfunc='sum'
)
# 로우(row) 별로 정규화: df.div(df.sum(axis='columns'), axis='index')
df_pivot = df_pivot.div(
    df_pivot.sum(axis='columns'), axis='index'
)
df_pivot.sort_values('F', inplace=True)
df_pivot.head()

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    color={'F': 'crimson', 'M': 'royalblue'},
    xlabel="",
    title="마포구 대여소별 이용자 성별 비율",
    figsize=(10, 15)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
ax.axvline(x=use_per_sex_rate['F'], color='grey'); # 마포구 여성 이용비율 41.3%

> [흠시] 아까 전, 위 '2.1 성별비율' 에서, 마포구 따릉이 이용의 성별비율은 남:녀 = 59:41 이라고 했다.  
가운데 회색선은 이 41를 나타낸다. 즉, 이 선이 일반적인 남녀비율의 평균(기준)이 된다.

> [흠시] 이 선(값 0.41) 을 기준으로 오른쪽으로 넘어간 빨간색 막대들 (여성 비율이 0.41 이상인 대여소)은 여성비율이 조금 더 많은 곳들이다.  
반대로, 이 선을 기준으로 왼쪽으로 넘어간 파란색 막대들 (남성 비울이 0.59 이상인 대여소)은 남성비율이 더 많은 곳들이다.

> [흠시] 한편, 이 선을 기준으로 위 조건에 해당하는 막대들은 ~정규분포 모양을 하고있음~을 짐작할 수 있다.

In [ ]:
# favor_f = pd.DataFrame({
#     'ratio': df_pivot['F'][df_pivot['F'] > 0.413] - 0.413,
#     'favor': 'F'
# })
favor_f = (df_pivot[df_pivot['F'] > use_per_sex_rate['F']]
    .__getitem__(['F']) # [['F']]
    .sub(use_per_sex_rate['F'])
    .rename(columns={'F': 'ratio'})
    .assign(favor='F')
)
favor_f.head()

In [ ]:
# favor_m = pd.DataFrame({
#     'ratio': 0.587 - df_pivot['M'][df_pivot['M'] > 0.587],
#     'favor': 'M'
# })
favor_m = (df_pivot[df_pivot['M'] > use_per_sex_rate['M']]
    .__getitem__(['M']) # [['M']]
    .sub(use_per_sex_rate['M'])
    .mul(-1)
    .rename(columns={'M': 'ratio'})
    .assign(favor='M')
)
favor_m.head()

In [ ]:
favor = pd.concat([favor_f, favor_m])
favor

`seaborn.kdeplot` https://seaborn.pydata.org/generated/seaborn.kdeplot.html
- Plot univariate or bivariate distributions using kernel density estimation.
- A kernel density estimate (KDE) plot is a method for visualizing the distribution of observations in a dataset, analagous to a histogram. KDE represents the data using a continuous probability density curve in one or more dimensions.

In [ ]:
# In seaborn.histplot,
#   stat='density' normalizes counts so that the area of the histogram is 1
#   stat='probability' normalizes counts so that the sum of the bar heights is 1

_, axes = plt.subplots(ncols=2, figsize=(15, 4))
sns.kdeplot(favor['ratio'], ax=axes[0])
sns.histplot(favor['ratio'], stat='density', kde=True, ax=axes[1])
for ax in axes:
    ax.set_frame_on(False)
    ax.set_ylabel("")

> [흠시] 단순히 0.41을 조금 넘는다고 이 대여소는 남녀비율 차이가 있다! 라고 말하기는 좀 오버스러운거 같으니,  
0.41과의 차이 즉, 편차가 2배이상 차이가 나는 대여소들만 비율 차이가 있다고 해석해보자.

In [ ]:
from scipy import stats

favor[np.abs(stats.zscore(favor['ratio'])) > 2]

> [흠시] 이렇게 2배이상 차이나는 대여소들은 다음과 같았다.  
> > 여자 비율이 유의미하게 더 높은 대여소들 (괄호 안의 값은 0.41 기준과의 차이)  
> > - 상암월드컵파크 9단지 앞 (0.20)  
> > - 망원초록길 입구 (0.16)   
> >   
> > 남자 비율이 유의미하게 더 높은 대여소들  
> > - 서강대 남문 옆 (0.18)  
> > - DMC빌 앞 (0.16)  
> > - 서강대 정문 건너편 (0.15)

> [흠시] 굳이 해석을 하자면, 서강대학교 근처 대여소들은 남자들이 더 많이 탄다는 정도?  
전체 73개의 대여소 중, 5개의 대여소만 유의미했으므로 (전체의 6%),   
일반적으로 대여소별 남녀 이용자 비율이 유의미하게 다르다고 보긴 힘들듯 하다.

### 3.2. 대여소별로 연령대비율이 많이 다를까?

> [흠시] 이번엔 연령대 기준으로 모든 대여소를 살펴본다.  
만약, 유의미하게 다르다면, 각 연령대별 사용비율이 높은 대여소들이 어딘지 살펴볼 수 있을 것이다.

In [ ]:
# 마포구 대여소별 연령대에 따른 이용비율을 구하자.

# pivot_table 명령을 수행할 때 결측치(NaN)가 나타나는 경우 `fill_value`로 값을 지정할 수 있다.
df_pivot = mapo_users.pivot_table(
    index='대여소명', columns='연령대코드', values='이용건수', aggfunc='sum', fill_value=0
)
df_pivot = df_pivot.div(
    df_pivot.sum(axis='columns'), axis='index'
)
df_pivot = df_pivot.reindex(columns=age_order).sort_values('20대')
df_pivot.head()

In [ ]:
ax = df_pivot.plot.barh(
    stacked=True,
    rot=0,
    xlabel="",
    title="마포구 대여소별 이용자 연령대 비율",
    figsize=(10, 15)
)
ax.set_frame_on(False)
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5));

> [흠시] 대여소 순서는 이용비율이 제일 많은 20대기준으로 정렬하였다.  
일단 이렇게보니, 대여소별 연령대비율은 달라보이긴 한다.  
아무래도, 성별은 전체량을 나누는 카테고리가 2개였고, 연령대는 7개이다 보니, 변동 폭도 훨씬 커보인다.

In [ ]:
from scipy import stats

_, ax = plt.subplots(figsize=(10, 4))
for i, (age, ds) in enumerate(df_pivot.iteritems()):
    sns.kdeplot(stats.zscore(ds), label=age, color=f'C{i}', ax=ax)
    sns.rugplot(stats.zscore(ds), ax=ax) # 디폴트 컬러 사용
ax.set_frame_on(False)
ax.set_ylabel("")
ax.legend(frameon=False);

> [흠시] 위 그래프는 마포구의 모든 대여소의 연령대 비율을 정규화한 분포를 나타낸 것이다.  
즉 x축에 있는 값 하나가, 한 대여소에서의 해당 연령의 비율을 나타낸다.  
(여기서는 물론 정규화해서 값 자체로는 의미가 없다. 여기서는 연령대별 분포 비교에 의미를 둔다.)

> [흠시] 보면, 20대의 경우 정규분포를 제일 정확히 그리고 있으며,  
10대, 40~60대의 경우 거의 같은 분포를 그린다.  
60대의 경우, 편차가 제일 적었다.

> [흠시] 즉, 20대는 대여소간에 편차가 좀 있다는 거고,   
70대~의 경우, 대여소간 편차가 별로 없다는 말이다. (즉 다 비율이 다 고만고만하다는 뜻)

> [흠시] 하지만, 아래 rug 를 보면, 70대에 속한 값 하나가 매우 극단적으로 높은 걸 알 수 있는데, 이는 특정 대여소가 다른 대여소들보다 압도적으로 사용되고 있음을 말한다.

> [흠시] 이렇게만 봐서는 뭐가 얼마나 어떻게 다른지 직관적으로 해석이 잘 안된다.  
또, 특정 연령대 비율이 높은 대여소가 어딘지 알 수가 없다.  
따라서, 연령대별로 비율이 높은 대여소들을 지도에 직접 시각화해서 봐보자.

In [ ]:
# 위에서는 대여소명을 인덱스로 pivot_table 명령을 수행했는데, 이번에는 대여소번호를 인덱스로 사용해보자.

df_pivot = mapo_users.pivot_table(
    index='대여소번호', columns='연령대코드', values='이용건수', aggfunc='sum', fill_value=0
)
df_pivot = df_pivot.div(df_pivot.sum(axis='columns'), axis='index')
df_pivot = df_pivot.reindex(columns=age_order)

# TypeError: cannot append a non-category item to a CategoricalIndex
df_pivot.columns = df_pivot.columns.astype('object')

# 대여소 정보 데이터(rental)에서 대여소번호에 해당하는 대여소명, 위도, 경도를 불러오자.
df_pivot = (df_pivot
    .merge(rental[['대여소명', '위도', '경도']], left_index=True, right_index=True)
    .sort_values('20대')
)
df_pivot.head()

In [ ]:
import folium, json

geo_path = 데이터_폴더 / 'seoul_municipalities_geo_simple.json'
with open(geo_path, encoding='utf-8') as fp:
    geo_str = json.load(fp)

mapo_center = df_pivot[['위도', '경도']].mean()

bike_map = folium.Map(
    location=mapo_center,
    zoom_start=13,
    zoom_control=False
)

folium.TileLayer(
    tiles='CartoDB positron',
    overlay=True
).add_to(bike_map)

# 마포구 지역 폴리곤 추가
for region_data in geo_str['features']:
    if region_data['properties']['SIG_KOR_NM'] == '마포구':
        polygon = np.array(region_data['geometry']['coordinates'])[0][:, (1, 0)] # fancy indexing
        folium.Polygon(polygon, color='darkgrey').add_to(bike_map)
        break
bike_map

> [흠시] 아래 시각화한 지도는 다음과 같이 보면 된다.  
> > ```
> > 1. 각 포인트는 대여소를 나타낸다.
> > 2. 색이 진할수록 이용비율이 높고, 옅을수록 낮음을 나타낸다.
> > ```

- `folium` 패키지(0.11.0)에서 사용하는 `branca` 패키지(0.4.1)에는 유니코드 문자를 제대로 처리하지 못하는 버그가 있다.
- 이를 해결하기 위해 `branca` 패키지를 깃헙에서 바로 다운로드한 후 설치하도록 한다.

```bash
conda install git
pip install git+https://github.com/python-visualization/branca@master
```

In [ ]:
age_color = {
    age: color for age, color in zip(
        age_order, ['blue', 'darkorange', 'green', 'red', 'purple', 'brown', 'magenta']
    )
}

dict_df_age = {}
for age in age_order:
    df_latlng_values = df_pivot[[age, '대여소명', '위도', '경도']].reset_index()
    
    # normalize data between 0 to 1.
    _min, _max = df_latlng_values[age].apply(['min', 'max'])
    df_latlng_values[age] = (df_latlng_values[age] - _min) / (_max - _min)
    
    # draw circle on each rental position.
    feature_group = folium.FeatureGroup(name=age, overlay=False)
    for _, ds in df_latlng_values.iterrows():
        folium.CircleMarker(ds[['위도', '경도']],
            radius=5,
            opacity=0,
            fill=True,
            fill_color=age_color[age],
            fill_opacity=ds[age],
            tooltip=f"[{ds['대여소명']}]\n{ds[age]:.2f}",
        ).add_to(feature_group)
    feature_group.add_to(bike_map)
    dict_df_age[age] = df_latlng_values[['대여소명', age]].sort_values(age, ascending=False)

folium.LayerControl(collapsed=False).add_to(bike_map)
bike_map

### - ~10대

> [흠시] \~10대의 경우, 특정 스팟 몇 군데가 주로 비율이 높다.  
다음과 같은 스팟들이다. (괄호 안의 값은 기존 연령별 이용비율을 0~1로 정규화한 값)  
> > 1. 공덕역 2번출구 (1.00)  
> > 2. 마포구청역 (0.90)  
> > 3. 마포 신수공원 앞 (0.87)    
> > 4. 상암월드컵파크 1단지 교차로 (0.84)

> [흠시] 이런 곳이 ~10대들이 이용하는 대여소 중, 인기가 있는 곳이라 할 수 있겠다.  
이 외, 그렇다할 눈에띄는 패턴은 보이지 않는다.

### - 20대

> [흠시] 20대의 경우, 확연히 신수동, **상수동, 그리고 신촌 쪽에 이용비율이 높은 것**을 알 수 있다.  
모두 대학가 및 대학생들의 주 활동지역임을 생각해보면, 상식적으로 이해가 간다.

### - 30대

> [흠시] 30대의 경우, 20대에게 인기가 많았던 **대학가쪽을 제외한 전 지역에 골고루 이용비율이 비슷함**을 보인다.  
**DMC, 망원동, 공덕동쪽**이 대표적인 곳들이다.  
20대와는 전혀 다른 패턴을 보이는게 재밌는 포인트다.

### - 40대

> [흠시] 40대의 경우, 패턴이 확실하다. DMC쪽에 대부분의 사용이 몰려있다.  
한편, 망원동, 신수동 등 20-30대들의 주 플레이스였던 곳에는 눈에띄게 사용비율이 적음을 알 수 있다.

### - 50대

> [흠시] 50대의 경우, **40대의 패턴이 비슷해보인다.** 즉, DMC쪽으로 이용비율이 쏠려있다.  
다만, 40대에서 높았던 몇 군데의 대여소의 이용비율이 조금 줄고, 공덕동 쪽에 40대보다 조금 더 많은 이용비율을 보인다.

### - 60대

> [흠시] 60대의 경우,**마포구청 일대 그리고 창천동 일대**에 주로 사용비율이 쏠려있다.  
아무래도 주택 주거지가 많은 지역들이다.

### - 70대~

> [흠시] 70대의 경우도, 패턴이 확실하다. DMC, 그것도 DMC 내 특정 지역에 몰려있다.  
나머지 대여소들의 이용비율은 고만고만하다.  
다른 패턴들에 비해 매우 뚜렷해서 인상적이다.

---
## 4. 어느 대여소간 따릉이 이동이 많을까?

> [흠시] 이번에는, 따릉이의 구체적인 트래픽에 대해 살펴본다.  
정말 구체적인 따릉이가 지나온 경로 데이터까지 있으면 좋겠지만, 아쉽게도 구체적인 경로는 없고,  
다만 어느 대여소에서 어느 대여소로 이동했는지 정도는 데이터에 있다.

> [흠시] 이를 러프하게 시각화하면 대략 다음과 같다.

![](k.kakaocdn.net_dn_coPnM5_btqwTj2eu8a_iTXgGu1BBnf6zhA72Iv6gK_img.png)

> [흠시] 언뜻 뭔가 보이기는 하지만, 선도 너무 많고 해석하기 힘든 시각화이므로, 다음을 기준으로 시각화해보려 한다.  
> > 1. 대여소는 점으로 표시한다.
> > 2. 대여소의 이용량(대여+반납량)은 투명도로 표현한다.
> > 3. 대여소 -> 대여소의 경로는 선으로 표현한다.
> > 4. 해당 경로를 이용한 량은 선의 투명도로 표현한다. 즉 짙으면 이용량이 많다는 뜻이다.
> > 5. 어느정도 이용량이 많은 경로(선)는 해당 경로의 방향을 표시한다. (이는 모든 경로에 화살표를 그리면 제대로 보기 힘들기 때문이다)
> > 6. 평일만 살펴본다. (주말은 제외)
> > 7. 시간 순으로 살펴볼 것인데, 모든 시간대를 다 그려서 포스팅하면, 너무 많으므로, 나름 의미를 부여한 시간순으로 살펴볼 것이다.

> [흠시] 이제 본격적으로 시간에 따라 살펴보자.

In [ ]:
from collections import namedtuple

def get_arrows(locations, color='blue', weight=1, size=6, n_arrows=3, some_map=None):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''

    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, opacity=weight,
                      radius=size, rotation=rotation).add_to(some_map))
    return arrows

def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

In [ ]:
import folium.plugins

def draw_traffic(df, dayofweek, time, region, marker_on=False, heatmap=False, arrow=True):
    if dayofweek == "평일":
        _dayofweek = range(0, 5)
    elif dayofweek == "주말":
        _dayofweek = range(5, 7)
    else:
        raise "dayofweek 인자를 평일 또는 주말로 입력해주세요."

    # 대여지역 또는 반납지역이 region인 데이터만 모은다.
    df = df[(df['대여지역'] == region) | (df['반납지역'] == region)]
    # 평일 또는 주말별로 데이터를 모은다.
    df = df[(df['대여요일'].isin(_dayofweek)) & (df['대여시간'].isin(time))]

    # 트래픽 라인 그리기
    by_rental = (df.groupby(['대여대여소번호', '반납대여소번호']).size()
        .rename('weight')
        .to_frame()
        .merge(rental[['위도', '경도']], left_on='대여대여소번호', right_index=True)
        .rename(columns={'위도': '대여대여소위도', '경도': '대여대여소경도'})
        .merge(rental[['위도', '경도']], left_on='반납대여소번호', right_index=True)
        .rename(columns={'위도': '반납대여소위도', '경도': '반납대여소경도'})
    )
    by_rental['weight'] /= by_rental['weight'].max() # normalize

    # - 특정 트래픽 라인 이용 분포 그리기
    _, ax = plt.subplots(figsize=(10, 4))
    sns.histplot(by_rental['weight'], stat='density', bins=140, linewidth=0, ax=ax)
    sns.rugplot(by_rental['weight'], ax=ax)
    ax.set_xlabel("이용 횟수 (normalized)")
    ax.set_ylabel("")
    ax.set_title("특정 경로 이용 분포도 (normalized)")

    # - 지도 먼저 그려주기.
    center = by_rental[['대여대여소위도', '대여대여소경도']].mean(axis=0)
    bike_map = folium.Map(
        location=center,
        zoom_start=13,
        zoom_control=False
    )

    folium.TileLayer(
        tiles='CartoDB positron',
        overlay=True
    ).add_to(bike_map)

    for _, ds in by_rental.iterrows():
        # [weight, 대여대여소위도, 대여대여소경도, 반납대여소위도, 반납대여소경도]
        _row = ds.to_numpy()
        weight = _row[0]
        # 화살표 추가
        if arrow and weight >= 0.3:
            # 화살표 다 그리면 렌더링이 힘드니, 0.3 이상만 그려준다.
            arrows = get_arrows(
                [_row[1:3], _row[3:5]], weight=weight, size=2, n_arrows=3, some_map=bike_map
            )
            for arrow in arrows:
                arrow.add_to(bike_map)

        line = _row[1:].reshape((2, 2)) # [(_row[2], _row[2]), (_row[3], _row[4])]
        folium.PolyLine(line, opacity=weight+0.05, weight=0.5).add_to(bike_map)

    # heatmap 그리기
    if heatmap:
        _df = (df[df['대여지역'] == region].groupby('대여대여소번호').size()
            .rename('weight')
            .to_frame()
            .merge(rental[['위도', '경도']], left_index=True, right_index=True)
        )
        _df['weight'] = _df['weight'] / _df['weight'].max()
        points = _df.to_numpy()[:, (1, 2, 0)] # 위도, 경도, weight
        folium.plugins.HeatMap(points, radius=15).add_to(bike_map)

    # marker 그리기
    if marker_on:
        _df = pd.DataFrame(data={
            '대여': df.groupby('대여대여소번호').size(),
            '반납': df.groupby('반납대여소번호').size()
        }).fillna(0)
        _df = _df.merge(rental, left_index=True, right_index=True)
        
        _df['이용'] = _df['대여'] + _df['반납']
        _df['비율'] = _df['이용'] / _df['이용'].max()

        for _, ds in _df.iterrows():
            folium.CircleMarker(ds[['위도', '경도']], 
                fill=True,
                fill_color='blue',
                fill_opacity=ds['비율'],
                radius=5,
                opacity=0,
                tooltip=f"[{ds['대여소명']}\n대여: {ds['대여']:.0f}\n반납: {ds['반납']:.0f}"
            ).add_to(bike_map)

    # 지역 폴리곤 추가.
    for region_data in geo_str['features']:
        if region_data['properties']['SIG_KOR_NM'] == region:
            polygon = np.array(region_data['geometry']['coordinates'])[0][:, (1, 0)] # fancy indexing
            folium.Polygon(polygon, color='darkgrey').add_to(bike_map)
            break
    
    return bike_map

### 4.1. 출근 시간대(7시~10시 사이)

In [ ]:
draw_traffic(df, "평일", range(7, 10), "마포구", marker_on=True)

> [흠시]  
> > DMC, 홍대입구역 2번 출구쪽 트래픽 활성화

> [흠시] 눈에 띄게 이동량이 많은 경로는 다음과 같았다.  
> - (연남동쪽 아파트 / 연희삼거리) -> 홍대입구역 2번 출구
> - DMC역 9번 출구 앞 -> (누리꿈 스케어 옆 / DMC산학협력 연구센터 옆)

> [흠시] 별다른 설명 없이, 위 시각화만 보고도, 출근시간 대에 주로 자전거 이동이 어떻게 움직이는지 알 수 있다.  
압도적인 출근지점은 홍대입구역과 DMC역 근처 일터라고 볼 수 있다.

### 4.2. 낮 시간대 (10시~17시 사이)

In [ ]:
draw_traffic(df, "평일", range(10, 17), "마포구", marker_on=True)

> [흠시]  
> > 홍대입구역 2번 출구 외 전체 트래픽 활성화

> [흠시] 출근 시간대보다 전반적인 대여소들이 비교적 활성화된 것을 알 수 있다.  
이동경로를 보면, (연남동쪽 아파트 <-> 홍대입구역 2번 출구) 이 제일 활발한 것을 알 수 있고,  
다른 경로들도 평균적으로 많이 활발해진 것을 알 수 있다.

### 4.3. 퇴근 시간대 (17시~20시 사이)

In [ ]:
draw_traffic(df, "평일", list(range(17, 20)), "마포구", marker_on=True)

> [흠시]  
> > 망원동, 합정동, 홍대입구역 2번 출구쪽 트래픽 활성화

> [흠시] 한강 바로 위에있는 망원동-합정동 라인의 대여소들이 유독 활발해졌다.  
한편, 화살표가 뜬 이동경로를 살펴보면, 출근했던 방향의 반대 방향으로 트래픽이 증가함을 알 수 있다.

### 4.3. 저녁 시간대 (20시~24시 사이)

In [ ]:
draw_traffic(df, "평일", list(range(20, 24)), "마포구", marker_on=True)

> [흠시]  
> > 망원동, 홍대입구역 2번 출구쪽 트래픽 활성화

> [흠시] 망원동쪽 대여소(마포구민 체육센터 앞)의 이용량이 지금까지 이용량 1위였던 홍대입구역을 제쳤다.  
이동경로는 이전만큼 뚜렷치 않다.

> [흠시] 아래 그래프는, 시간에 따른 대여소 이용량 분포를 정규화한 분포다.  
즉, x 축 값 하나하나는 해당 시간대의 한 대여소의 이용량을 나타낸다. (정규화하여 수치는 의미없음.)

In [ ]:
# 평일 마포구 대여소별 대여량 및 반납량 분포 (normalized)

from scipy import stats

df_weekdays = df[df['대여요일'].isin(range(1, 6))]

times = [range(7, 10), range(10, 17), range(17, 20), range(20, 24), range(0, 7)]

_, axes = plt.subplots(ncols=2, figsize=(15, 4))

for ax, use in zip(axes, ['대여', '반납']):
    for i, time in enumerate(times):
        by_rental = df_weekdays[
            (df_weekdays[f'{use}지역'] == '마포구') & (df_weekdays[f'{use}시간'].isin(time))
        ]
        by_rental = by_rental.groupby(f'{use}대여소번호').size()
        
        by_rental = stats.zscore(by_rental)
        by_rental = (by_rental - by_rental.min()) / (by_rental.max() - by_rental.min())
        
        sns.kdeplot(by_rental, label=f"{time[0]}시-{time[-1]+1}시", color=f"C{i}", ax=ax)
        sns.rugplot(by_rental, ax=ax)
        
    ax.set_frame_on(False)
    ax.set_ylabel("")
    ax.set_title(f"마포구 내 대여소 {use}량 분포(정규화)")
    ax.legend(frameon=False)

In [ ]:
# 평일 마포구 대여소별 대여량+반납량 분포 (normalized)

_, ax = plt.subplots(figsize=(10, 4))

for i, time in enumerate(times):
    by_rental = df_weekdays[
        (df_weekdays['대여지역'] == '마포구') & (df_weekdays['대여시간'].isin(time))
    ].groupby('대여대여소번호').size()
    by_return = df_weekdays[
        (df_weekdays['반납지역'] == '마포구') & (df_weekdays['반납시간'].isin(time))
    ].groupby('반납대여소번호').size()
    
    by_rental = (by_rental + by_return).fillna(0).rename('이용량')
    by_rental = stats.zscore(by_rental)
    by_rental = (by_rental - by_rental.min()) / (by_rental.max() - by_rental.min())
    
    sns.kdeplot(by_rental, label=f"{time[0]}시-{time[-1]+1}시", color=f"C{i}", ax=ax)
    sns.rugplot(by_rental, ax=ax)

ax.set_frame_on(False)
ax.set_ylabel("")
ax.set_title(f"마포구 내 대여소 이용량 분포(정규화)")
ax.legend(frameon=False);

> [흠시] 먼저, 이 그래프를 통해서 대여소의 이용량이 전체적으로 어떻게 바뀌어나가는지 알 수 있다.

> [흠시] 이 분포는 0~1로 정규화되어있다. 각 시간대별로 항상 0과 1이 기준이 되는 값들이 존재하고 그 사이에 분포가 어떻게 형성되어있는지를 알아보려는 것이다. 한편, 대여소의 이용량 분포를 통해, 트래픽이 어떻게 형성되어있는지 짐작할 수도 있다.

> [흠시] 먼저, 출근시간대(7시~10시)를 보면, 왼쪽으로 굉장히 skewed 하다. 즉, 이용량이 몇몇 대여소로 쏠려있다는 것을 말한다.  
위에서 살펴봤던 대로, '홍대입구역 2번 출구 앞' 대여소나, 'DMC역' 대여소가 이렇게 사용량이 쏠려있는 대여소다.  
이 몇몇 대여소의 사용량이 다른 대여소의 사용량보다 압도적임을 나타낸다.  
트래픽 역시, 이 대여소들로 쏠려있음을 짐작할 수 있다.

> [흠시] 한편, 10시-17시 사이가 되면 이런 skewed 가 조금 덜 해진다.  
즉 이전 시간대보다 몇몇 대여소의 사용량이 압도하는 현상이 줄어들었다는 것이다.  
위 시각화된 지도에서 보면, 대여소들의 전반적인 트래픽이 활성화 되었는데, 이와도 일맥상통한다고 볼 수 있다.  
이와 비슷한 현상은 20-24시에도 동일하게 등장하고, 나머지 시간대도 비슷하다고 볼 수 있다.

> [흠시] 혹시나 놓치는 부분이 있을까봐, 각 대여소별 이용량 히트맵도 시각화해본다.  
굳이 해석하진 않고, 다르게 시각화하여도 위에서 해석한 내용과 일치한다는 것을 보여주기 위한 일종의 참고자료로 남겨둔다.

In [ ]:
by_rental = \
    df_weekdays[df_weekdays['대여지역'] == '마포구'].pivot_table(
        index='대여대여소명', columns='대여시간', aggfunc='size', fill_value=0
    ) + \
    df_weekdays[df_weekdays['반납지역'] == '마포구'].pivot_table(
        index='반납대여소명', columns='반납시간', aggfunc='size', fill_value=0
    )

_, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(by_rental, cmap='Blues', cbar=False, ax=ax)
ax.set_xlabel("시간")
ax.set_ylabel("");

---
## 5. 어느 지역으로 유입/유출이 많을까?

> [흠시] 마포구로 들어오고 나가는 따릉이들 중, 어느 지역과 가장 트래픽량이 많을까?

In [ ]:
# 먼저 대여지역 또는 반납지역이 마포구인 데이터를 모아보자.

mapo_df = df[(df['대여지역'] == '마포구') | (df['반납지역'] == '마포구')]

In [ ]:
# 유출: 마포구 -> 반납지역

out_counts = mapo_df['반납지역'].value_counts()
out_counts

In [ ]:
# 유입: 대여지역 -> 마포구

in_counts = mapo_df['대여지역'].value_counts()
in_counts

In [ ]:
inout_counts = pd.DataFrame(data=
    {'유출': out_counts.drop('마포구'), '유입': in_counts.drop('마포구')}
)
inout_counts.sort_values('유출', ascending=False, inplace=True)
ax = inout_counts.plot.bar(
    rot=0,
    color={'유출': 'C3', '유입': 'C2'},
    title="유출, 유입 지역",
    figsize=(15, 4)
)
ax.set_frame_on(False)
ax.legend(frameon=False);

> [흠시] 유출-유입이 선형적인 관계에 있음을 알 수 있다. 즉, 유출이 많은 지역이 유입도 많다.  
한편, 상식적인 얘기겠지만, 역시나 인근 지역으로의 트래픽이 많다.  
서대문 > 영등포 > 은평구 > 영등포 순이고, 모두 마포구를 둘러싸고 있는 인근지역들이다.

> [흠시] 지도로 시각화하면 다음과 같다.

In [ ]:
bike_map = folium.plugins.DualMap(
    location=[37.541, 126.986],
    zoom_start=10,
    zoom_control=False,
    tiles='CartoDB positron'
)
folium.Choropleth(geo_data=geo_str,
    data=out_counts.drop('마포구'),
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Reds').add_to(bike_map.m1)
folium.Choropleth(geo_data=geo_str,
    data=in_counts.drop('마포구'),
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Greens').add_to(bike_map.m2)
bike_map

---
## 정리

사용량이 가장 많은 지역 중 하나인 마포를 살펴보았다.  
발견한 내용을 하나씩 정리하면,  

1. 근처 지역에서 출퇴근이 어느정도 활발한 지역이다. 
2. 출근 시간에, 외부 유입이 약 30-35% 정도, 퇴근 시간에는 외부 유출이 25-30% 정도 된다. 
3. 출퇴근 시간에 DMC역, 연남동, 홍대입구역을 중심으로 트래픽량이 활발하다.      
4. 저녁시간대에는 한강을 끼고있는 망원동을 중심으로 트래픽이 활성화된다.
5. 모든 시간 통틀어, 아무래도 번화가의 중심지인 홍대입구역 앞이 압도적으로 활발하긴 하다.
6. 한편, 이동경로가 핫한 곳은 2~3군데로 고정되어 있다.
7. 외부 트래픽은 서대문 > 영등포 > 은평구 > 영등포 순으로 활발하고, 모두 이웃지역이다.